In [101]:
import yfinance as yf
import numpy as np
import pandas as pd
from get_data_yf import get_symbols

#TODO writes advised company recommendation data to an excel file

RECOMMENDATION_WEIGHTS = [2, 1, 0, -1, -2] #weights for strongBuy, buy, hold, sell,	strongSell advisors respectively
TIMEZONE_WEIGHTS = [4, 3, 2, 1] # weights for 0, -1, -2, -3rd months respectively
ADVICE_COUNT = 10 #how many of best scored symbols we want to get data

In [104]:
symbols = get_symbols()
symbol = yf.Ticker('XTLB')
recommendations = symbol.recommendations
recommendations = recommendations.drop('period', axis='columns')
recommendations

,strongBuy,buy,hold,sell,strongSell
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0


In [98]:
def create_weigth_kernel(vertical_weights, horizontal_weights):
    kernel_values = np.outer(vertical_weights, horizontal_weights)
    kernel = pd.DataFrame(kernel_values)
    kernel.rename(columns={0:'strongBuy', 1:'buy', 2:'hold', 
                              3:'sell', 4:'strongSell'}, inplace = True)
    
    return kernel

In [107]:
def recommendation_score(recommendations, weight_kernel):
    sum = recommendations.to_numpy().sum()
    if sum == 0:
        return None
    weighted_sum = (weight_kernel * recommendations).to_numpy().sum()
    return weighted_sum / sum 

In [99]:
def evaluate_symbols(symbols, timezone_weights, recommendation_weights):
    #
    scores = {}
    weight_kernel = create_weigth_kernel(timezone_weights, recommendation_weights)
    
    for symbol in symbols:
        recommendations = yf.Ticker(symbol).recommendations
        recommendations = recommendations.drop('period', axis='columns')
        
        score = recommendation_score(recommendations, weight_kernel)
        if score:  
            scores[symbol] = score
        else: 
            print(f"No advisors found for {symbol}.") 
    return scores  

In [102]:
scores = evaluate_symbols(symbols, ADVICE_COUNT, TIMEZONE_WEIGHTS, RECOMMENDATION_WEIGHTS)
print(scores)

C:\Users\vlns\AppData\Local\Temp\ipykernel_20936\3781186824.py:4: RuntimeWarning: invalid value encountered in scalar divide
  return weighted_sum / sum
C:\Users\vlns\AppData\Local\Temp\ipykernel_20936\3781186824.py:4: RuntimeWarning: invalid value encountered in scalar divide
  return weighted_sum / sum


{'AAPL': 2.426829268292683, 'MSFT': 2.918781725888325, 'XWEL': 2.5, 'YJ': -2.5, 'YHGJ': nan, 'ZEUS': 1.375, 'XTLB': nan, 'IBM': 0.9647058823529412, 'NFLX': 2.0172413793103448, 'V': 2.9177215189873418, 'CRM': 2.5591397849462365, 'DIS': 1.9606299212598426, 'TSLA': 0.6168831168831169, 'BA': 2.495575221238938, 'AMZN': 3.0651162790697675, 'INTC': 0.6358381502890174, 'BABA': 3.1391752577319587, 'HD': 2.1805555555555554, 'PYPL': 2.094972067039106, 'NKE': 1.9411764705882353, 'XOM': 1.0, 'CSCO': 1.644859813084112, 'MRK': 2.3653846153846154, 'UNH': 3.1067961165048543}
